### Скачиваем библиотеку, необходимую для инференса + Делаем импорты. Были небольшие проблемы с установкой pyg, но в коллабе со строчкой ниже всё должно запускаться, единственное что, возможно нужно будет перезапустить сессию

In [ ]:
!pip install pyg-lib -f https://data.pyg.org/whl/torch-2.4.0+cu121.html

In [ ]:
!gdown 1_RQxjVFlve12NMTSyvEWlL6m9eylHmxU
!gdown 12NVI9hbSnFjfiT27d-FkaeLmkz3WgvwI
!gdown 1jkpplWIKV6IS7AHBPEPcgmiQC8T6RuuJ
!gdown 1V9X-iDGABK0njxTm6nVJPDH-cquHqJ8s
!gdown 1fXx465_ICgmZ1-9_Sl45w7GHLHzndyRo

In [ ]:
%pip install torch-geometric

In [ ]:
# other
from itertools import combinations, groupby
from tqdm import tqdm

# for graphs
from torch_geometric.utils import from_networkx
from torch_geometric.data import Batch

import networkx as nx
import torch_geometric.transforms as T
from torch_geometric.utils import from_networkx

# for data
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# for visualazing
import matplotlib.pyplot as plt
import seaborn as sns

#sklearn
import sklearn

## Стандартная предобработка графа

In [ ]:
data_path = '/content'
graph = pd.read_csv(f'{data_path}/graph.csv').drop(columns=['Unnamed: 0'])
# train = pd.read_csv(f'{data_path}/train_X.csv')

In [ ]:
graph_inversed = graph.copy()
tmp = graph_inversed['contractor_id1']
graph_inversed['contractor_id1'] = graph_inversed['contractor_id2']
graph_inversed['contractor_id2'] = tmp
graph_inversed['base_graph'] = 1
graph['base_graph'] = 0

In [ ]:
graph_joined = pd.concat([graph, graph_inversed], axis=0)

In [ ]:
graph_joined = graph_joined.drop_duplicates(subset=['contractor_id1', 'contractor_id2', 'Distance'])
graph = graph_joined[graph_joined['base_graph'] == 1].drop(columns='base_graph')

## Предобрабатываем табличные данные

In [ ]:
def preprocess_data(train, test, delete_corr_features=True, scale=True, init_features2drop=None):

    ts_features2drop = ['agg_cec_requests__g_contract__total_sum_accepted__all__sum__6W', 'agg_cec_requests__g_contract__total_sum_accepted__all__sum__8W', 'agg_cec_requests__g_contract__total_sum_accepted__all__sum__5W', 'agg_all_contracts__g_contract__rel_change_price_last_ds__isMain__last__ALL_TIME', 'agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__last__ALL_TIME', 'agg_all_contracts__g_contract__bit_da_guid__isMain__count__ALL_TIME', 'agg_all_contracts__g_contract__abs_change_price_last_ds__isMain__mean__ALL_TIME', 'agg_cec_requests__g_contract__request_id__all__count__8W', 'agg_cec_requests__g_contract__request_id__all__count__4W', 'agg_cec_requests__g_contract__request_id__all__count__5W', 'agg_cec_requests__g_contract__request_id__all__count__7W', 'agg_ArbitrationCases__g_contractor__DefendantSum__sum__ALL_TIME', 'agg_FinanceAndTaxesFTS__g_contractor__TaxesSum__last__ALL_TIME', 'agg_FinanceAndTaxesFTS__g_contractor__TaxPenaltiesSum__last__ALL_TIME', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__ALL_TIME', 'counteragent_sum_agg_ks2__g_contract__total_sum__all__sum__ALL_TIME', 'counteragent_sum_agg_cec_requests__g_contract__request_id__all__count__ALL_TIME', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__ALL_TIME', 'agg_materials__g_contract__order_id__countDistinct__ALL_TIME', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__12M', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__5M', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__6M', 'agg_cec_requests__g_contract__total_sum_accepted__all__sum__ALL_TIME', 'counteragent_sum_agg_cec_requests__g_contract__total_sum_accepted__all__sum__ALL_TIME', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__7M', 'agg_ks2__g_contract__id__all__count__ALL_TIME', 'agg_cec_requests__g_contract__request_id__all__count__12W', 'agg_ks2__g_contract__total_sum__all__sum__ALL_TIME', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__8M', 'specialization_sum_agg_ks2__g_contract__total_sum__all__sum__ALL_TIME', 'counteragent_sum_agg_payments__g_contract__sum__all__sum__ALL_TIME', 'specialization_sum_agg_payments__g_contract__sum__all__sum__ALL_TIME', 'agg_workers__g_contract__fact_workers__all__mean__4W', 'agg_workers__g_contract__fact_workers__all__mean__3W', 'agg_workers__g_contract__fact_workers__all__mean__26W', 'agg_workers__g_contract__fact_workers__all__mean__6W', 'agg_workers__g_contract__fact_workers__all__mean__12W', 'agg_workers__g_contract__fact_workers__all__mean__8W', 'agg_workers__g_contract__fact_workers__all__mean__5W', 'agg_Finance__g_contractor__Value__CostPrice_y__last__ALL_TIME', 'agg_Finance__g_contractor__Value__NetProfit_y__last__ALL_TIME', 'agg_spass_applications__g_contract__appl_count_week__mean__6W', 'agg_spass_applications__g_contract__appl_count_week__mean__26W', 'agg_spass_applications__g_contract__appl_count_week__max__ALL_TIME', 'agg_spass_applications__g_contract__appl_count_week__mean__12W', 'agg_spass_applications__g_contract__appl_count_week__mean__ALL_TIME', 'agg_payments__g_contract__sum__all__countDistinct__4W', 'agg_spass_applications__g_contract__appl_count_week__mean__4W', 'agg_payments__g_contract__sum__all__countDistinct__2W', 'agg_scontrol__g_contractor__close_delay__defect_type_app__mean__ALL_TIME', 'agg_ks2__g_contract__id__all__count__2W', 'agg_ks2__g_contract__id__all__count__1W', 'agg_cec_requests__g_contract__total_sum_accepted__all__sum__4W', 'agg_ks2__g_contract__id__all__count__4W', 'agg_FinanceAndTaxesFTS__g_contractor__Income__last__ALL_TIME', 'agg_tender_proposal__g_contractor__id__ALL__countDistinct__8W', 'agg_tender_proposal__g_contractor__id__ALL__countDistinct__26W', 'agg_tender_proposal__g_contractor__id__ALL__countDistinct__52W', 'agg_payments__g_contract__sum__all__countDistinct__12W', 'agg_payments__g_contract__sum__all__sum__8W', 'agg_materials__g_contract__order_id__countDistinct__12W', 'agg_materials__g_contract__material_id__countDistinct__ALL_TIME', 'agg_materials__g_contract__order_id__countDistinct__8W', 'agg_payments__g_contract__sum__all__countDistinct__ALL_TIME', 'agg_payments__g_contract__date__advance__min__ALL_TIME', 'agg_payments__g_contract__sum__all__sum__ALL_TIME', 'agg_ks2__g_contract__total_sum__all__sum__8W', 'agg_ks2__g_contract__id__all__count__12W', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__3M', 'agg_cec_requests__g_contract__time_btw_requests__all__mean__4M', 'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12M', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12M', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_24M', 'agg_ArbitrationCases__g_contractor__PlaintiffSum__sum__12_48M', 'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_36M', 'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_48M', 'agg_ArbitrationCases__g_contractor__DefendantSum__sum__12_24M', 'agg_Finance__g_contractor__Value__CurrentAssets__last__ALL_TIME', 'agg_Finance__g_contractor__Value__Balance__last__ALL_TIME', 'agg_Finance__g_contractor__Value__Capital__last__ALL_TIME', 'agg_workers__g_contract__fact_workers__all__mean__1W', 'agg_sroomer__g_contractor__sroomer_id__count__12M', 'agg_sroomer__g_contractor__sroomer_id__count__6M', 'agg_sroomer__g_contractor__sroomer_id__count__ALL_TIME', 'agg_sroomer__g_contractor__sroomer_id__count__3M', 'agg_ks2__g_contract__total_sum__all__sum__12W', 'agg_cec_requests__g_contract__created_dt__accepted__min__ALL_TIME', 'agg_payments__g_contract__sum__all__countDistinct__1W']
    constant_features_names = ['agg_BoardOfDirectors__g_contractor__Name__count__ALL_TIME', 'agg_ConsolidatedIndicator__g_contractor__Index__IndexOfDueDiligence__mean__ALL_TIME', 'agg_spark_extended_report__g_contractor__CreditLimitSum__last__ALL_TIME', 'agg_ConsolidatedIndicator__g_contractor__Index__Overall__mean__ALL_TIME', 'agg_ConsolidatedIndicator__g_contractor__Index__PaymentIndex__mean__ALL_TIME', 'agg_spark_extended_report__g_contractor__CompanySizeRevenue__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__PledgeeActiveCount__last__ALL_TIME', 'contract_date', 'contract_init_sum', 'agg_spark_extended_report__g_contractor__PledgerActiveCount__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__PledgeeCeasedCount__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__PledgerCeasedCount__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__EstimatedNetLiabilitiesSum__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__EstimatedClaimsSum__last__ALL_TIME', 'agg_spark_extended_report__g_contractor__EstimatedLiabilitiesSum__last__ALL_TIME', 'agg_ConsolidatedIndicator__g_contractor__Index__FailureScore__mean__ALL_TIME']

    if delete_corr_features:
        train = train.drop(columns=ts_features2drop)
        test = test.drop(columns=ts_features2drop)

    # for now lets drop columns where num of nans is greater that  a half of df length
    isna_stat = train.isna().sum()[train.isna().sum() > 0]
    features2drop = [col for col in isna_stat.index if isna_stat[col] > (train.shape[0] * (3/5))]
    train.drop(columns=features2drop, inplace=True)
    test.drop(columns=features2drop, inplace=True)

    # other we`ll fill with median
    for col in isna_stat.index:
        if col not in features2drop:
            imp = SimpleImputer(strategy="median").fit(train[col].values.reshape(-1, 1))
            train[col] = imp.transform(train[col].values.reshape(-1, 1))
            test[col] = imp.transform(test[col].values.reshape(-1, 1))

    #translate to timestamp
    dates_columns = train.select_dtypes('object').columns

    new_dates_cols = []

    for date_col in dates_columns:
        train[f"{date_col}_day"] = train[date_col].apply(lambda x: datetime.fromisoformat(x).day)
        train[f"{date_col}_month"] = train[date_col].apply(lambda x: datetime.fromisoformat(x).month)

        test[f"{date_col}_day"] = test[date_col].apply(lambda x: datetime.fromisoformat(x).day)
        test[f"{date_col}_month"] = test[date_col].apply(lambda x: datetime.fromisoformat(x).month)

        train[date_col] = train[date_col].apply(lambda x: datetime.fromisoformat(x).timestamp())
        test[date_col] = test[date_col].apply(lambda x: datetime.fromisoformat(x).timestamp())

        new_dates_cols.extend([f"{date_col}_day", f"{date_col}_month"])

    if init_features2drop is not None:
      train = train.drop(columns=init_features2drop)
      test = test.drop(columns=init_features2drop)

    return train, test, constant_features_names

In [ ]:
root_data_path = "/content"

X_train = pd.read_csv(f"{root_data_path}/train_X.csv")
y_train_all = pd.read_csv(f"{root_data_path}/train_y.csv")

train = X_train.merge(y_train_all, on=["contract_id", "report_date"], how="left")
test = pd.read_csv(f"{root_data_path}/test2_X.csv")

y_train_all = train["default6"]
train_orig = train.copy()

###  Добавляем сгенерированные графовые фичи (contractor_graph_features.csv - первая версия)

In [ ]:
# adding contractor new graph features
graph_features = pd.read_csv(f"{root_data_path}/contractor_graph_features.csv")
train = train.merge(graph_features, on="contractor_id", how="left")
test = test.merge(graph_features, on="contractor_id", how="left")

FileNotFoundError: [Errno 2] No such file or directory: '/content/contractor_graph_features.csv'

In [ ]:
for_drop = ["project_id", "building_id"]
cat_features = ["specialization_id"]

train, test, constant_features_names = preprocess_data(
    train.copy(),
    test.copy(),
    delete_corr_features=False,
    scale=False,
    init_features2drop=for_drop,
)

## Тут представлена не совсем нужная часть про генерацию фичей для нод(по-сути контрактора) - это используется при supervised обучении, в другом ноутбуке

In [ ]:
def gen_features_for_contractor(df, best_cols):

  contractor_features_agg = df.groupby(
      'contractor_id', as_index=False
  )[list(best_cols)].agg(['mean', 'std']).fillna(0)

  contractor_features_agg.columns = ['contractor_id'] + [f'{col[0]}_{col[1]}' for col in contractor_features_agg.columns[1:]]

  contractor_features_pro_dict = dict(
    reports_count=('contract_id', 'count'),
    contracts_count=('contract_id', lambda x: len(np.unique(x))),
    contract_len_mean=('contract_id', lambda x: x.value_counts().mean()),
    contract_len_std=('contract_id', lambda x: x.value_counts().std()),
  )

  contractor_features_pro = df.groupby('contractor_id', as_index=False).agg(
      **contractor_features_pro_dict
  ).fillna(0)

  contractor_features = contractor_features_agg.merge(
      contractor_features_pro,
      on='contractor_id',
      how='left'
  )

  targets = None

  return contractor_features, targets

In [ ]:

contractor_features_train, _ = gen_features_for_contractor(
    train,
    [
        'report_date',
        'contract_init_sum',
        'contract_date',
        'contract_current_sum',
    ] + [col for col in list(train.columns[6:]) if col != 'default6']
)
contractor_features_test, _ = gen_features_for_contractor(
    test,
    [
        'report_date',
        'contract_init_sum',
        'contract_date',
        'contract_current_sum',
    ] + [col for col in list(train.columns[6:]) if col != 'default6']
)
contractor_features = pd.concat(
  [
      contractor_features_train,
      contractor_features_test
  ],
  axis=0
).drop_duplicates(subset=['contractor_id'])
graph_contract_ids = list(set(graph['contractor_id1'].unique()) | set(graph['contractor_id2'].unique()))
contractor_features = contractor_features[
    contractor_features['contractor_id'].isin(graph_contract_ids)
]

In [ ]:
contractor_features["contractor_id_index"] = contractor_features["contractor_id"]
contractor_features = contractor_features.set_index('contractor_id_index').to_dict(orient="index")

<ipython-input-16-14e46fc281f0>:11: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  contractor_features = contractor_features.set_index('contractor_id_index').to_dict(orient="index")


## Тут мы уже собираем граф и переводим его в формат Torch Geometric

In [ ]:
graph = graph[
    (
      graph['contractor_id1'].isin(list(contractor_features.keys()))
    ) & (
      graph['contractor_id2'].isin(list(contractor_features.keys()))
    )
]

(286795, 3)

In [ ]:
G = nx.Graph()

pairs = []
for _, row in tqdm(graph.iterrows()):
  pairs.append((int(row['contractor_id1']), int(row['contractor_id2']), int(row['Distance'])))

G.add_weighted_edges_from(pairs)

286795it [00:22, 12708.80it/s]


In [ ]:
nx.set_node_attributes(G, contractor_features)

In [ ]:
to_keep_as_single = ['contractor_id', 'train_mask']

g_data = from_networkx(
  G,
  group_node_attrs=[
      x for x in list(next(iter(G.nodes(data=True)))[-1].keys()) \
      if x not in to_keep_as_single
  ],
)

## Конструируем Графовую модель в парадигме Inductive Unsupervised learning

In [ ]:
import os
import pickle
import random
from glob import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GraphSAGE
from torch_geometric.utils import to_networkx
from torch_geometric.loader import LinkNeighborLoader

import sys

In [ ]:
class train_CFG:
    def __init__(
        self,
    ):
        pass

    def __call__(self, attr, value):
        setattr(self, attr, value)

def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
set_seed(69)

In [ ]:
class cfg:
    def __init__(self, ):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.seed = 111
        self.graph_num_nodes = g_data.x.shape[0]

cfg = cfg()

set_seed(cfg.seed)

g_data = g_data.to(cfg.device, "x", "edge_index")

### Тут функции для трейна, так же мы с каким-то интервалом обучаем бустинг, что смотреть на качество эмбеддингов

In [ ]:
%pip install catboost -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00


In [ ]:
import catboost

def train_one_epoch_GraphSage(loader, model, optimizer, reduction="mean"):

    model.train()

    total_loss = 0.0

    stream = tqdm(loader, leave=False)

    cur_count = 0

    for batch in stream:

        batch = batch.to(cfg.device)
        h = model(batch.x, batch.edge_index)

        h_src = h[batch.edge_label_index[0]]
        h_dst = h[batch.edge_label_index[1]]
        pred = (h_src * h_dst).sum(dim=-1)
        loss = F.binary_cross_entropy_with_logits(pred, batch.edge_label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        cur_count += pred.size(0)

        # print(loss.item()
        total_loss += loss.item() * pred.size(0)
        stream.set_description(f"Cur loss: {total_loss / cur_count}")

    return total_loss / cfg.graph_num_nodes

def eval_one_epoch_catboost(
    model,
    dataset,
    return_hidden=False,
    catboost_train_set=None,
    catboost_eval_set=None
):

    model.eval()

    with torch.no_grad():
        out = model(dataset.x, dataset.edge_index).cpu()


    clf = catboost.CatBoostClassifier(
        iterations=500,
        task_type="CPU",
        border_count=254,
        random_state=cfg.seed,
        eval_metric="NormalizedGini",
        auto_class_weights='Balanced',
        use_best_model=True,
        early_stopping_rounds=30,
    )

    embeddings = pd.DataFrame(
        out.numpy().tolist(), columns=[f'emb_{i}' for i in range(out.shape[1])]
    )
    embeddings['contractor_id'] = dataset['contractor_id'].numpy().tolist()

    catboost_train_set = catboost_train_set.merge(
        embeddings,
        how='left',
        on='contractor_id'
    ).fillna(-1)
    catboost_train_set['embeddings'] = catboost_train_set.loc[
        :, embeddings.columns[1:]
    ].values.tolist()

    catboost_eval_set = catboost_eval_set.merge(
        embeddings,
        how='left',
        on='contractor_id'
    ).fillna(-1)
    catboost_eval_set['embeddings'] = catboost_eval_set.loc[
        :, embeddings.columns[1:]
    ].values.tolist()

    # print(catboost_train_set.isna().sum())

    train_pool = catboost.Pool(
        data=catboost_train_set.drop(columns=['default6']),
        label=catboost_train_set['default6'],
        embedding_features=['embeddings']
    )

    # print(catboost_eval_set.isna().sum())
    eval_pool = catboost.Pool(
        data=catboost_eval_set.drop(columns=['default6']),
        label=catboost_eval_set['default6'],
        embedding_features=['embeddings']
    )

    clf.fit(
        train_pool,
        verbose=25,
        eval_set=[train_pool, eval_pool],
    )

    val_preds = clf.predict(eval_pool)

    val_score = 2 * sklearn.metrics.roc_auc_score(eval_pool.get_label(), val_preds.reshape(-1)) - 1

    print('CATBOOST VAL SCORE: ', val_score)

    if return_hidden:
        return val_score, out
    else:
        return val_score



def train_embedder(
    model=None,
    loader=None,
    optimizer=None,
    train_cfg=None,
    full_dataset=None,
    earlystopper_loss=None,
    catboost_train_set=None,
    catboost_eval_set=None
):

    stream = tqdm(range(train_cfg.num_epochs))

    val_gini = 0

    max_val_gini = 0

    for epoch in stream:

        #training
        loss = train_one_epoch_GraphSage(
            loader=loader,
            model=model,
            optimizer=optimizer,
            reduction="mean"
        )

        if epoch % 1 == 0:

            val_gini, h = eval_one_epoch_catboost(
                model,
                full_dataset,
                return_hidden=True,
                catboost_train_set=catboost_train_set,
                catboost_eval_set=catboost_eval_set
            )

        print(f"train loss: {round(loss, 5)}")
        stream.set_description(f"train loss: {round(loss, 5)}, val gini: {val_gini}")

        if val_gini > max_val_gini:
          max_val_gini = val_gini
          earlystopper_loss.save_checkpoint(max_val_gini, model)

        earlystopper_loss(loss, model)

### Архитектура модели основана на графовой свертке - GraphSAGE, а само обучение - по сути contrastive learning

In [ ]:
set_seed(cfg.seed)


# GraphSage Pro Implementation
class GraphSagePro(nn.Module):


    def __init__(
        self,
        n_in=128,
        n_out=1,
        hidden_dim=128,
        n_layers=3
    ):

        super(GraphSagePro, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(n_in, hidden_dim),
            nn.ReLU(),
            nn.LayerNorm(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
        )
        self.conv = GraphSAGE(in_channels=hidden_dim, hidden_channels=hidden_dim * 4, out_channels=hidden_dim * 2, num_layers=n_layers)
        self.decoder = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LeakyReLU(negative_slope=0.1),
            nn.Linear(hidden_dim, n_out),
        )

    def forward(self, x, edge_index):

        x = self.encoder(x)
        x = self.conv(x, edge_index)

        return self.decoder(x)

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for skip in self.skips:
            skip.reset_parameters()

In [ ]:
class EarlyStoppingContrastiveLoss:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, loss, model):

        score = loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(loss, model)
        elif score > self.best_score - self.delta:
            self.counter += 1
            if self.verbose:
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(loss, model)
            self.counter = 0

    def save_checkpoint(self, best_score, model):
        if self.verbose:
            self.trace_func(f'Contrastive Loss decreased from ({self.loss_min:.4f} to {self.best_score:.4f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.loss_min = best_score

### Настройки обучения

In [ ]:
set_seed(cfg.seed)
#inductive setting
train_loader = LinkNeighborLoader(
    g_data,
    batch_size=32768,
    shuffle=True,
    neg_sampling_ratio=2.0,
    num_neighbors=[-1, -1, -1],
)

model = GraphSagePro(
    n_in=g_data.num_features,
    n_out=36,
    hidden_dim=256,
    n_layers=4
).to(cfg.device)

earlystopper_loss = EarlyStoppingContrastiveLoss(
    patience=100,
    verbose=False,
    delta=0.01,
    path="/content/best_model.pt",
    trace_func=print,
)

optimizer = torch.optim.NAdam(model.parameters(), lr=3e-5, weight_decay=1e-6)

class train_cfg:
    def __init__(self):
        self.num_epochs = 20

In [ ]:
contracts_ids = train['contract_id'].unique()

train_size = int(len(contracts_ids) * 0.8)
val_size = 1 - train_size
set_seed(cfg.seed)
train_ids = np.random.choice(contracts_ids, size=train_size, replace=False, )
val_ids = [id for id in contracts_ids if id not in train_ids]

train_set = train[train.contract_id.isin(train_ids)]
val_set = train[train.contract_id.isin(val_ids)]

### Запускаем обучение

In [ ]:
set_seed(cfg.seed)

train_embedder(
    model=model,
    loader=train_loader,
    optimizer=optimizer,
    train_cfg=train_cfg(),
    full_dataset=g_data,
    earlystopper_loss=earlystopper_loss,
    catboost_train_set=train_set,
    catboost_eval_set=val_set
)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

Learning rate set to 0.092993
0:	test: 0.4700721	test1: 0.3311749	best: 0.3311749 (0)	total: 69.9ms	remaining: 34.9s
25:	test: 0.8735839	test1: 0.5079436	best: 0.5192083 (19)	total: 1.38s	remaining: 25.2s
50:	test: 0.9393981	test1: 0.5382189	best: 0.5382189 (50)	total: 2.73s	remaining: 24s
75:	test: 0.9668431	test1: 0.5488954	best: 0.5551491 (57)	total: 4.66s	remaining: 26s
100:	test: 0.9817513	test1: 0.5620894	best: 0.5688994 (88)	total: 6.95s	remaining: 27.5s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5688993502
bestIteration = 88

Shrink model to first 89 iterations.
CATBOOST VAL SCORE:  0.3819871301483879
train loss: 1565.72328


  0%|          | 0/18 [00:00<?, ?it/s]

Learning rate set to 0.092993
0:	test: 0.4882245	test1: 0.3939266	best: 0.3939266 (0)	total: 65.2ms	remaining: 32.5s
25:	test: 0.8594431	test1: 0.5082218	best: 0.5119572 (23)	total: 1.39s	remaining: 25.4s
50:	test: 0.9322879	test1: 0.5362177	best: 0.5362177 (50)	total: 3.44s	remaining: 30.3s
75:	test: 0.9650535	test1: 0.5425047	best: 0.5425047 (75)	total: 5.63s	remaining: 31.4s
100:	test: 0.9815157	test1: 0.5621120	best: 0.5621120 (100)	total: 6.91s	remaining: 27.3s
125:	test: 0.9890045	test1: 0.5718615	best: 0.5718615 (125)	total: 8.16s	remaining: 24.2s
150:	test: 0.9936339	test1: 0.5730518	best: 0.5746185 (134)	total: 9.48s	remaining: 21.9s
175:	test: 0.9960302	test1: 0.5644332	best: 0.5748539 (163)	total: 10.8s	remaining: 19.9s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5748538941
bestIteration = 163

Shrink model to first 164 iterations.
CATBOOST VAL SCORE:  0.3856400021479587
train loss: 1561.28246


...

Learning rate set to 0.092993
0:	test: 0.4877557	test1: 0.3853887	best: 0.3853887 (0)	total: 69.6ms	remaining: 34.7s
25:	test: 0.8678832	test1: 0.5410464	best: 0.5567209 (21)	total: 1.4s	remaining: 25.5s
50:	test: 0.9386733	test1: 0.5475665	best: 0.5567209 (21)	total: 2.69s	remaining: 23.7s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5567208728
bestIteration = 21

Shrink model to first 22 iterations.
CATBOOST VAL SCORE:  0.42294422467646364
train loss: 1561.26789


  0%|          | 0/18 [00:00<?, ?it/s]

Learning rate set to 0.092993
0:	test: 0.4549080	test1: 0.3482362	best: 0.3482362 (0)	total: 64.3ms	remaining: 32.1s
25:	test: 0.8594272	test1: 0.5338133	best: 0.5338133 (25)	total: 1.46s	remaining: 26.5s
50:	test: 0.9355068	test1: 0.5390190	best: 0.5492446 (48)	total: 2.76s	remaining: 24.3s
75:	test: 0.9654624	test1: 0.5581725	best: 0.5616641 (70)	total: 4.09s	remaining: 22.8s
100:	test: 0.9797109	test1: 0.5708043	best: 0.5708204 (98)	total: 5.41s	remaining: 21.4s
125:	test: 0.9891587	test1: 0.5753426	best: 0.5804527 (104)	total: 6.71s	remaining: 19.9s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5804526823
bestIteration = 104

Shrink model to first 105 iterations.
CATBOOST VAL SCORE:  0.395219002273256
train loss: 1561.24285


  0%|          | 0/18 [00:00<?, ?it/s]

Learning rate set to 0.092993
0:	test: 0.4789560	test1: 0.3238341	best: 0.3238341 (0)	total: 69.4ms	remaining: 34.6s
25:	test: 0.8631199	test1: 0.4895517	best: 0.5133771 (10)	total: 1.4s	remaining: 25.6s
Stopped by overfitting detector  (30 iterations wait)

bestTest = 0.5133770831
bestIteration = 10

Shrink model to first 11 iterations.
CATBOOST VAL SCORE:  0.35230413303023256
train loss: 1561.20876


### Подгружаем лучшую модельку и получаем эмбеддинги

In [ ]:
best_model = GraphSagePro(
    n_in=g_data.num_features,
    n_out=36,
    hidden_dim=256,
    n_layers=4
).to(cfg.device)

best_model.load_state_dict(
    torch.load('/content/best_model.pt')
)

<ipython-input-46-9c5d8ddf9c94>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('/content/best_model.pt')


<All keys matched successfully>

In [ ]:
with torch.no_grad():
    out = best_model(g_data.x, g_data.edge_index).cpu()

In [ ]:
c_ids = g_data.contractor_id

In [ ]:
embeds_df = pd.DataFrame(
    {
        "contractor_id" : c_ids.cpu(),
    }
)

embeds_df = pd.concat(
    [embeds_df,
    pd.DataFrame(out.detach().cpu(), columns=[f'embed_{i}' for i in range(out.shape[1])])],
    axis=1
)

In [ ]:
embeds_df.to_parquet('unsupervised_embeddings.parquet', index=False)